<a href="https://colab.research.google.com/github/djsg2021utec/TESIS_MAESTRIA/blob/main/Implementaci%C3%B3n_del_ViVIT_RWF_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## Importar librerías
from tqdm import tqdm
import time
import os
from base64 import b64encode
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import HTML
from base64 import b64encode
from matplotlib import pyplot as plt
import glob
from random import shuffle
import tensorflow as tf
import timeit
import shutil
import sys

In [2]:
import tensorflow as tf
import timeit
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Datos RWF-2000

In [3]:
# Enlace del repositorio donde está alojandos los datos
# https://github.com/mchengny/RWF2000-Video-Database-for-Violence-Detection

#%pip install gdown (Descomentar si requiere instalar)

# ********************************************************************
# Descargando los fragmentos del archivo ZIP de la data RWF-2000.zip
# ********************************************************************
download_links = [
    "https://drive.google.com/uc?id=1nQ9IR3cGc4NEDOXhPQ89id8je8Uj2VUc",
    "https://drive.google.com/uc?id=1w9G_Z7gkXZzK4DImdI8wanyjs22fQARO",
    "https://drive.google.com/uc?id=15LhjavoUsLS01CPkc3qav0rJxBc9d4nl"
]

for link in tqdm(download_links, desc="Descargando fragmentos"):
    !gdown {link}
# ********************************************************************
# Juntando los fragmentos del archivo ZIP de la data RWF-2000.zip
# ********************************************************************
!cat RWF-2000.zip.001 RWF-2000.zip.002 RWF-2000.zip.003 > RWF-2000.zip

# Eliminando los fragmentos
for fragment in tqdm(["RWF-2000.zip.001", "RWF-2000.zip.002", "RWF-2000.zip.003"], desc="Eliminando fragmentos"):
    !rm /content/{fragment}

# Descomprimiendo los archivos en el directorio RWF-2000
!unzip "/content/RWF-2000.zip" -d "/content/"

# Eliminando RWF-2000.zip
!rm /content/RWF-2000.zip

# Asegurar que la memoria RAM se liberó
import gc
gc.collect()

time.sleep(10)

Descargando fragmentos:   0%|          | 0/3 [00:00<?, ?it/s]

Downloading...
From: https://drive.google.com/uc?id=1nQ9IR3cGc4NEDOXhPQ89id8je8Uj2VUc
To: /content/RWF-2000.zip.001
100% 4.29G/4.29G [00:42<00:00, 102MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w9G_Z7gkXZzK4DImdI8wanyjs22fQARO
To: /content/RWF-2000.zip.002
100% 4.29G/4.29G [00:45<00:00, 94.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=15LhjavoUsLS01CPkc3qav0rJxBc9d4nl
To: /content/RWF-2000.zip.003
100% 3.74G/3.74G [00:51<00:00, 73.1MB/s]


Eliminando fragmentos:   0%|          | 0/3 [00:00<?, ?it/s]

Archive:  /content/RWF-2000.zip
   creating: /content/RWF-2000/
   creating: /content/RWF-2000/train/
   creating: /content/RWF-2000/train/Fight/
  inflating: /content/RWF-2000/train/Fight/-1l5631l3fg_0.avi  
  inflating: /content/RWF-2000/train/Fight/-1l5631l3fg_1.avi  
  inflating: /content/RWF-2000/train/Fight/-1l5631l3fg_2.avi  
  inflating: /content/RWF-2000/train/Fight/0H2s9UJcNJ0_0.avi  
  inflating: /content/RWF-2000/train/Fight/0H2s9UJcNJ0_2.avi  
  inflating: /content/RWF-2000/train/Fight/0H2s9UJcNJ0_3.avi  
  inflating: /content/RWF-2000/train/Fight/0H2s9UJcNJ0_4.avi  
  inflating: /content/RWF-2000/train/Fight/0H2s9UJcNJ0_5.avi  
  inflating: /content/RWF-2000/train/Fight/0lHQ2f0d_0.avi  
  inflating: /content/RWF-2000/train/Fight/0lHQ2f0d_1.avi  
  inflating: /content/RWF-2000/train/Fight/0lHQ2f0d_2.avi  
  inflating: /content/RWF-2000/train/Fight/0lHQ2f0d_3.avi  
  inflating: /content/RWF-2000/train/Fight/0NWz-01A2yk_0.avi  
  inflating: /content/RWF-2000/train/Fight/0NWz

In [4]:
# ******************************************************************
# Se crea un dataframe para manejar fácilmente los datos de RWF-2000
# ******************************************************************
carpeta = 'RWF-2000'
lista_carpetas_avi = [
    {'ruta': f'/content/{carpeta}/train/Fight', 'data': 'train', 'etiqueta':'Fight'},
    {'ruta': f'/content/{carpeta}/train/NonFight', 'data': 'train', 'etiqueta':'NonFight'},
    {'ruta': f'/content/{carpeta}/val/Fight', 'data': 'val', 'etiqueta':'Fight'},
    {'ruta': f'/content/{carpeta}/val/NonFight', 'data': 'val', 'etiqueta':'NonFight'}
]
lista_archivos_avi = []
for carpeta in lista_carpetas_avi:
  ruta_carpeta=carpeta['ruta']
  tipo_data =carpeta['data']
  etiqueta_data =carpeta['etiqueta']
  for filename in os.listdir(ruta_carpeta):
      if filename.endswith(".avi"):
        registro_archivo = {'ruta': f'{ruta_carpeta}/{filename}', 'data': f'{tipo_data}', 'etiqueta':f'{etiqueta_data}'}
        lista_archivos_avi.append(registro_archivo)

In [5]:
videos_RWF2000_df = pd.DataFrame(lista_archivos_avi)
videos_RWF2000_df.describe()

,ruta,data,etiqueta
count,2000,2000,2000
unique,2000,2,2
top,/content/RWF-2000/train/Fight/trhfd_1019.avi,train,Fight
freq,1,1600,1000


In [6]:
# **********************************************************************
# Se crea carpetas para almacenar los archivos mp4
# **********************************************************************
carpeta = 'RWF-2000-mp4'
lista_carpetas_mp4 = [
    {'ruta': f'/content/{carpeta}/train/Fight', 'data': 'train', 'etiqueta':'Fight'},
    {'ruta': f'/content/{carpeta}/train/NonFight', 'data': 'train', 'etiqueta':'NonFight'},
    {'ruta': f'/content/{carpeta}/val/Fight', 'data': 'val', 'etiqueta':'Fight'},
    {'ruta': f'/content/{carpeta}/val/NonFight', 'data': 'val', 'etiqueta':'NonFight'}
]

for carpeta in lista_carpetas_mp4:
  os.makedirs(carpeta['ruta'], exist_ok=True)

In [6]:
# **********************************************************************
# Se toma una muestra con una fracción de los datos del dataframe
# **********************************************************************

# Función para obtener un subconjunto aleatorio del DataFrame
def grupo_de_muestras(group, frac=0.2):
    return group.sample(frac=frac)

# Dividir el DataFrame según las variables 'etapa' y 'etiqueta' y aplicar la función grupo_de_muestras
muestra_RWF2000_df = videos_RWF2000_df.groupby(['data', 'etiqueta']).apply(grupo_de_muestras).reset_index(drop=True)
# Se elimina una variable que tiene muchas variables
del videos_RWF2000_df

muestra_RWF2000_df.describe()

,ruta,data,etiqueta
count,400,400,400
unique,400,2,2
top,/content/RWF-2000/train/Fight/GKsvH_pVNQU_0.avi,train,Fight
freq,1,320,200


In [7]:
muestra_RWF2000_df.groupby(['data', 'etiqueta']).count().reset_index()

,data,etiqueta,ruta
0,train,Fight,160
1,train,NonFight,160
2,val,Fight,40
3,val,NonFight,40


In [8]:
# ******************************************************************
# Convertir a MP4 los videos AVI
# ******************************************************************
import gc

# Inicializar lista y DataFrame vacíos
lista_archivos_mp4 = []
df=muestra_RWF2000_df

# Procesar cada fila del DataFrame
for index, row in df.iterrows():
    ruta_archivo_avi = row['ruta']
    nombre_archivo = ruta_archivo_avi.split('/')[-1].split('.')[0]
    tipo_data = row['data']
    etiqueta = row['etiqueta']

    ruta_archivo_mp4 = f'/content/RWF-2000-mp4/{tipo_data}/{etiqueta}/{nombre_archivo}'
    registro_archivo = {'ruta': f'{ruta_archivo_mp4}.mp4', 'data': tipo_data, 'etiqueta': etiqueta}

    #os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input=ruta_archivo_avi, output=ruta_archivo_mp4))

    lista_archivos_mp4.append(registro_archivo)
    # if os.path.exists(f'{ruta_archivo_mp4}'):
    #   print(f"Se creó {ruta_archivo_mp4}.mp4 (data: {tipo_data}, etiqueta: {etiqueta})")
    #   # Libera memoria
    del ruta_archivo_avi, nombre_archivo, tipo_data, etiqueta, ruta_archivo_mp4, registro_archivo
    gc.collect()

In [9]:
RWF2000_mp4_df = pd.DataFrame(lista_archivos_mp4)
RWF2000_mp4_df.describe()

,ruta,data,etiqueta
count,400,400,400
unique,400,2,2
top,/content/RWF-2000-mp4/train/Fight/GKsvH_pVNQU_...,train,Fight
freq,1,320,200


### Implementación del ViViT

In [12]:
%%capture
%pip install tensorflow_docs
%pip install tensorflow
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [10]:
train_df=RWF2000_mp4_df[RWF2000_mp4_df['data']=='train']
test_df = RWF2000_mp4_df[RWF2000_mp4_df['data']=='val']

In [11]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 224

EPOCHS = 15

In [13]:
resize_and_rescale_layer = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

def resize_and_rescale(frame):
  with tf.device('/device:GPU:0'):
    resized_rescaled = resize_and_rescale_layer(frame[None, ...])
    resized_rescaled = resized_rescaled.numpy().squeeze()
    return resized_rescaled

In [18]:
def load_video(path, max_frames=0):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = resize_and_rescale(frame)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [19]:
def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

In [20]:
#PRUEBAS
feature_extractor = build_feature_extractor()

29084464/29084464 [==============================] - 2s 0us/step


In [21]:
# Label preprocessing with StringLookup.
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["etiqueta"]), mask_token=None
)
print(label_processor.get_vocabulary())

['Fight', 'NonFight']


In [22]:
import time
def volver_crear_mp4(ruta):
  path = ruta
  if os.path.exists(ruta):
    print("Se elimina el archivo: ",path)
    os.remove(ruta)
    time.sleep(4)
  root_destino = os.path.dirname(path)
  root_origen = root_destino.replace('/RWF-2000-mp4/','/RWF-2000/')
  nombre=path.split('/')[-1:][0].split('.')[0]
  ruta_archivo_avi=f'{root_origen}/{nombre}.avi'
  ruta_archivo_mp4=f'{root_destino}/{nombre}'

  os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input=ruta_archivo_avi, output=ruta_archivo_mp4))
  if os.path.exists(f'{ruta_archivo_mp4}.mp4'):
    print("Se vuelve a crear el archivo: ",path)
    time.sleep(10)

def prepare_all_videos(df):
    num_samples = len(df)
    video_paths = df["ruta"].values.tolist()
    labels = df["etiqueta"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(path))
        print(f'{idx} {path}:',"frame dimension",frames.shape,"\n")
        # Volver a generar videos en mp4 de aquellos que fallaron
        while frames.shape[0]==0:
          volver_crear_mp4(path)
          frames = load_video(os.path.join(path))
          print(f'{idx} {path}:',"frame dimension",frames.shape,"\n")
        # Pad shorter videos.
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            frames = np.concatenate((frames, padding), axis=0)

        frames = frames[None, ...]

        # Initialize placeholder to store the features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_features[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_features.squeeze()

    return frame_features, labels


In [23]:
train_data, train_labels = prepare_all_videos(train_df)
test_data, test_labels = prepare_all_videos(test_df)

Se han truncado las últimas 5000 líneas del flujo de salida.
301 /content/RWF-2000-mp4/train/NonFight/dP20avzQAf8_0.mp4: frame dimension (0,) 

301 /content/RWF-2000-mp4/train/NonFight/dP20avzQAf8_0.mp4: frame dimension (0,) 

301 /content/RWF-2000-mp4/train/NonFight/dP20avzQAf8_0.mp4: frame dimension (0,) 

Se vuelve a crear el archivo:  /content/RWF-2000-mp4/train/NonFight/dP20avzQAf8_0.mp4
301 /content/RWF-2000-mp4/train/NonFight/dP20avzQAf8_0.mp4: frame dimension (150, 224, 224, 3) 

1/1 [==============================] - 0s 35ms/step
302 /content/RWF-2000-mp4/train/NonFight/vwQvNOlX_0.mp4: frame dimension (150, 224, 224, 3) 

1/1 [==============================] - 0s 33ms/step
303 /content/RWF-2000-mp4/train/NonFight/dNDOYLTf_0.mp4: frame dimension (0,) 

303 /content/RWF-2000-mp4/train/NonFight/dNDOYLTf_0.mp4: frame dimension (0,) 

303 /content/RWF-2000-mp4/train/NonFight/dNDOYLTf_0.mp4: frame dimension (0,) 

303 /content/RWF-2000-mp4/train/NonFight/dNDOYLTf_0.mp4: frame dimens

### Cargar el 20\% de la data tarda 1h 11 min

In [25]:
## GUARDANDO LOS DATOS DE TRAIN.DF
np.save('train_data_RWF200.npy', train_data)
np.save('train_labels_RWF200.npy', train_labels)

## GUARDANDO LOS DATOS DE TEST.DF
np.save('test_data_RWF200.npy', test_data)
np.save('test_labels_RWF200.npy', test_labels)


In [57]:
train_data.shape

(320, 20, 1024)

In [ ]:

# Convertir el array de NumPy a un tensor de TensorFlow
train_data_tf = tf.convert_to_tensor(train_data)
train_labels_tf = tf.convert_to_tensor(train_labels)
test_data_tf = tf.convert_to_tensor(test_data)
test_labels_tf = tf.convert_to_tensor(test_labels)

print(train_data_tf)

In [47]:
#DATA
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (10, 224, 224, 3)
NUM_CLASSES = 11

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 60

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

In [48]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader

In [37]:
trainloader = prepare_dataloader(train_data_tf, train_labels_tf, "train")

In [49]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

In [50]:
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

In [51]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [53]:
trainloader = prepare_dataloader(train_data_tf, train_labels_tf, "train")
validloader = prepare_dataloader(test_data_tf, test_labels_tf, "valid")
testloader = prepare_dataloader(test_data_tf, test_labels_tf, "valid")

In [54]:
def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    # Train the model.
    _ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)

    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model


model = run_experiment()

Epoch 1/60


ValueError: ignored